In [1]:
from retrieval import SparseRetrieval, DenseRetrieval
from transformers import AutoTokenizer
from datasets import load_from_disk
import pickle

## Retrieval 클래스
- tokenizer, data_path, context_path를 인자로 가집니다.
- tokenizer -> 사용할 tokenizer를 사용합니다. DenseRetrieval의 경우 학습에 사용한 tokenizer를 사용해야합니다.
- context_path -> 가져올 corpus의 path를 인자로 넘겨주어야 합니다.
- data_path -> 가져온 문서나 embedding을 저장할 위치(미정)

# Retrieval 클래스의 method
- get_topk_doc_id_and_score : query에 대해서 top k개의 유사도가 높은 wiki id와 score를 가지고 옵니다.
- get_topk_doc_id_and_score_for_querys : query list 대해서 top k개의 유사도가 높은 wiki id와 score를 query - id list, query - score list 의 dict 형태로 전달해 줍니다.

In [12]:
valid_dataset = load_from_disk('/opt/ml/data/test_dataset/validation').to_pandas()
querys = valid_dataset['question']

## SparseRetrieval(elastic search) 예제

In [13]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base') # 토크나이저
elastic = SparseRetrieval(tokenizer=tokenizer) 
# 엘라스틱 서치는 nori 토크나이저를 사용하지만 exception이 생길 경우 bert tokenizer를 사용하는 
# BM25Okapi를 사용합니다.

Token indices sequence length is longer than the specified maximum sequence length for this model (1131 > 512). Running this sequence through the model will result in indexing errors
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [14]:
print(querys[0])

유령'은 어느 행성에서 지구로 왔는가?


In [15]:
# get_topk_doc_id_and_score를 호출하면 해당 query에 대해 유사도가 높은 wiki 문서의 id와 그 점수를 top k 개 만큼 가져옵니다.
doc_ids, scores = elastic.get_topk_doc_id_and_score(querys[0],5)
print(doc_ids)
print(scores)

[43280, 47081, 35064, 24024, 42242]
[20.487246, 20.10372, 19.851677, 19.529999, 18.778242]


In [16]:
# context를 확인 하고 싶으면 클래스 변수인 wiki_id_context_dict을 사용하면 됩니다.
print(elastic.wiki_id_context_dict[doc_ids[0]])

목성의 대기에서 보이는 줄무늬는 적도와 평행하면서 행성을 둘러싸는 대(zone)와 띠(belt)라고 불리는 물질의 반대 순환류에 의한 것이다. 대는 밝은 줄무늬로, 대기에서 상대적으로 고도가 높은 곳에 있다. 이들은 내부의 상승 기류를 가지고 있는 고기압 영역이다. 띠는 어두운 줄무늬로, 대기에서 상대적으로 고도가 낮은 곳에 있으며, 내부의 하강 기류를 가진다. 이들은 저기압 영역이다. 이러한 구조는 지구 대기의 고기압 및 저기압 세포와 어느정도 유사하나, 국지 작은 기압 세포와 상반되는 행성 전체를 둘러싸는 위도 줄무늬로서 매우 다른 구조를 가지고 있다. 이는 행성의 빠른 자전과 근본적인 대칭으로 인한 결과로 보인다. 행성에는 국지적인 가열을 일으키는 바다나 육지가 없으며 자전 속도는 지구보다 훨씬 빠르다. 행성에는 서로 다른 크기와 색상을 갖는 점과 같은 작은 구조들이 있다. 목성에서, 그러한 특색 중에서 가장 유명한 것은 대적점으로, 적어도 300년 동안 존재해 왔다. 이러한 구조의 실체는 거대한 폭풍이다. 그러한 점 중에 일부는 적란운이기도 하다.


In [17]:
# get_topk_doc_id_and_score_for_querys를 호출하고 인자로 query가 담긴 리스트와 top k를 인자로 넘겨주면
# query와 높은 유사도를 가진 wiki id 리스트와 score 리스트를 dict로 전달해 줍니다.
query_ids, query_scores = elastic.get_topk_doc_id_and_score_for_querys(querys.to_list(),5)

print(query_ids[querys[0]])
print(query_scores[querys[0]])

print(query_ids[querys[1]])
print(query_scores[querys[1]])

  0%|          | 0/600 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
100%|██████████| 600/600 [00:13<00:00, 45.72it/s]


[43280, 47081, 35064, 24024, 42242]
[20.487246, 20.10372, 19.851677, 19.529999, 18.778242]
[20081, 23179, 15556, 13590, 18470]
[23.784315, 21.420332, 21.085941, 20.766212, 19.601896]


## DenseRetrieval(elastic search) 예제

In [19]:
dense_retrieval = DenseRetrieval(tokenizer, 'p_encoder/','q_encoder/')
# query encoder, context encoder를 학습시키고 사용한 모델의 tokenizer와 모델의 저장 위치를 인자로 가집니다.

In [7]:
# 문서를 가져오는 function은 Retrieval과 동일합니다. (SparseRetrieval 참고)
ids, scores = dense_retrieval.get_topk_doc_id_and_score(querys[0],10)

In [8]:
print(ids)
print(scores)

[9727, 9257, 49262, 9615, 8988, 29581, 29064, 9671, 50628, 11628]
[319.8592529296875, 319.40545654296875, 319.20556640625, 318.966552734375, 318.74884033203125, 318.7379150390625, 318.4833068847656, 318.36279296875, 318.1867980957031, 318.1792907714844]


In [9]:
q_ids, q_scores = dense_retrieval.get_topk_doc_id_and_score_for_querys(querys.to_list(),10)

100%|██████████| 600/600 [00:00<00:00, 1795.60it/s]


In [21]:
print(q_ids[querys[0]])
print(q_scores[querys[0]])
print()
print(q_ids[querys[1]])
print(q_scores[querys[1]])

[14294, 13824, 53846, 14182, 13555, 34153, 33635, 14238, 55213, 16195]
[319.8595886230469, 319.4053955078125, 319.20562744140625, 318.9667663574219, 318.7488708496094, 318.73809814453125, 318.4835205078125, 318.3629150390625, 318.1866760253906, 318.1793212890625]

[14238, 16310, 53846, 29851, 13555, 51707, 38386, 35983, 50310, 27693]
[332.8433837890625, 332.751220703125, 332.5208435058594, 332.357177734375, 332.3047180175781, 332.29180908203125, 332.10577392578125, 332.10296630859375, 332.01019287109375, 331.9410400390625]


In [23]:
print(dense_retrieval.wiki_id_context_dict[ids[0]])

그라브 지역 (Graves)은 보르도 남부, 가론 강 좌안에 길게 위치하고 다음과 같은 AOC로 이뤄져 있다. * 페사크레오냥 (Pessac-Léognan AOC)은 보르도 외곽 지역근처에 위치한다. ::유명와인 : 샤토 오브리옹 (Chateau haut-brion, 1등급), 샤토 라 미시옹 오브리옹 (chateau la mission haut-brion), 샤토 파프 클르망 (chateau pape clement), 샤토 오브리앙 블랑 (chateau haut-brion blanc, 백) * 세롱 (Cérons AOC)은 그라브 지역 남부에 위치한 작은 지역이다. ::유명와인 : 샤토 갈랑 세롱 (Chateau gallant cerons, 백) * 그라브 (Graves AOC) 와 그라브 쉬페리외르 (Graves supérieurs AOC)는 나머지 그라브 지역에 사용한다.
